In [ ]:
import numpy as np
from numpy import array
from numpy import vstack
import json
import os
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import LSTM, Dense,Dropout, LayerNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from sklearn.model_selection import train_test_split

%load_ext tensorboard

In [ ]:
frames_changable = 20

In [ ]:
coordinates = open("/content/gdrive/MyDrive/MASTERS_THESIS/yolov7pose/vid_coordsOG.txt").read()

In [ ]:
coordinates2 = open("/content/gdrive/MyDrive/MASTERS_THESIS/yolov7pose/vid_coords.txt").read()

#Original Footage

In [ ]:
coordinates = coordinates.replace("\\n", "")

In [ ]:
vid_list = coordinates.split("{")
vid_list = vid_list[1:]

In [ ]:
data_dict = {}
dict_list = []

In [ ]:
for i in range(len(vid_list)):
  data_dict = {}

  vidname_coord_split = vid_list[i].split(", 'coords': ")
  vidname = vidname_coord_split[0]
  vidname = vidname.replace("\'vidname\': ", "")
  data_dict["video"] = vidname

  coords = vidname_coord_split[1]
  coords = coords.replace("[array(","")
  coords = coords.replace(")]","")
  coords = list(coords.split("],")) # number of frames in video
  # print(len(coords)) # number of frames in video
  for j in range(len(coords)):
    coords[j]  = list(coords[j].split(","))
    coords[j][0] = coords[j][0][2:]
    coords[j] = [x.strip(']]}""') for x in coords[j]]
    coords[j] = [x.strip(' ') for x in coords[j]]
    coords[j] = [float(value) for value in coords[j]]
    # removes the z value 
    del coords[j][3-1::3]
    # print(coords[j])
    
  data_dict["coords"] = coords

  dict_list.append(data_dict)

In [ ]:
len(dict_list)

In [ ]:
frame_len = []
for i in range(len(dict_list)):
  frame_len.append(len(dict_list[i]['coords']))

In [ ]:
count = 0
for i in range(len(dict_list)):
  frame_max = len(dict_list[i]['coords'])
  # print(frame_max)
  if frame_max > frames_changable+1:
    cut_off = frame_max - (frames_changable + 2)
    dict_list[i]['coords'] = dict_list[i]['coords'][cut_off+1:-1]
  # elif frame_max == 18:
  #   dict_list[i]['coords'] = dict_list[i]['coords'][:-1]
  elif frame_max < frames_changable:
    count +1
    # dict_list[i]['coords'] = [val for val in dict_list[i]['coords'] for _ in (0, 1)]
    # if len(dict_list[i]['coords']) > 16:
    #   cut_off = len(dict_list[i]['coords']) - 17
    #   dict_list[i]['coords'] = dict_list[i]['coords'][cut_off:-1]
print("total number of videos omitted afer ")
print(count)

In [ ]:
final = []
labels = []

for a in range(len(dict_list)):
  adjusted_final = {}
  if len(dict_list[a]['coords']) == frames_changable:
    coord = dict_list[a]['coords']
    vid = dict_list[a]['video']
    adjusted_final['coords'] = coord
    adjusted_final['video'] = vid
    adjusted_final['label'] = vid[1]
    final.append(adjusted_final)
    labels.append(vid[1])

In [ ]:
len(final)

In [ ]:
len(final[0]['coords'])

In [ ]:
coord_df = pd.DataFrame(final)

In [ ]:
frame_length_listdf = list("F%d" % i for i in range(1,frames_changable+1))


In [ ]:
coord_df[frame_length_listdf] = pd.DataFrame(coord_df.coords.tolist(), index= coord_df.index)

In [ ]:
coord_df

#Mirrored Footage

In [ ]:
coordinates2 = coordinates2.replace("\\n", "")

In [ ]:
vid_list2 = coordinates2.split("{")
vid_list2 = vid_list2[1:]

In [ ]:
len(vid_list2)

In [ ]:
vid_list2[0]

In [ ]:
data_dict2 = {}
dict_list2 = []

In [ ]:
for i in range(len(vid_list2)):
  data_dict2 = {}

  vidname_coord_split2 = vid_list2[i].split(", 'coords': ")
  vidname2 = vidname_coord_split2[0]
  vidname2 = vidname2.replace("\'vidname\': ", "")
  data_dict2["video"] = vidname2

  coords2 = vidname_coord_split2[1]
  coords2 = coords2.replace("[array(","")
  coords2 = coords2.replace(")]","")
  coords2 = list(coords2.split("],")) # number of frames in video
  # print(len(coords)) # number of frames in video
  for j in range(len(coords2)):
    coords2[j]  = list(coords2[j].split(","))
    coords2[j][0] = coords2[j][0][2:]
    coords2[j] = [x.strip(']]}""') for x in coords2[j]]
    coords2[j] = [x.strip(' ') for x in coords2[j]]
    coords2[j] = [float(value) for value in coords2[j]]
    # removes the z value 
    del coords2[j][3-1::3]
    # print(coords[j])
    
  data_dict2["coords"] = coords2

  dict_list2.append(data_dict2)

In [ ]:
len(dict_list2)

In [ ]:
frame_len2 = []
for i in range(len(dict_list2)):
  frame_len2.append(len(dict_list2[i]['coords']))

In [ ]:
count2 = 0
for i in range(len(dict_list2)):
  frame_max2 = len(dict_list2[i]['coords'])
  # print(frame_max)
  if frame_max2 > frames_changable+1:
    cut_off = frame_max2 - (frames_changable+2)
    dict_list2[i]['coords'] = dict_list2[i]['coords'][cut_off+1:-1]
  # elif frame_max2 == 18:
  #   dict_list2[i]['coords'] = dict_list2[i]['coords'][:-1]
  elif frame_max2 < frames_changable:
    count2 +1
    # dict_list[i]['coords'] = [val for val in dict_list[i]['coords'] for _ in (0, 1)]
    # if len(dict_list[i]['coords']) > 16:
    #   cut_off = len(dict_list[i]['coords']) - 17
    #   dict_list[i]['coords'] = dict_list[i]['coords'][cut_off:-1]
print("total number of videos omitted afer ")
print(count2)

In [ ]:
len(dict_list2[0])

In [ ]:
final2 = []
labels2 = []

for a in range(len(dict_list2)):
  adjusted_final2 = {}
  if len(dict_list2[a]['coords']) == frames_changable:
    coord2 = dict_list2[a]['coords']
    vid2 = dict_list2[a]['video']
    adjusted_final2['coords'] = coord2
    adjusted_final2['video'] = vid2
    # print(vid2)
    adjusted_final2['label'] = vid2[2]
    final2.append(adjusted_final2)
    labels2.append(vid2[2])

In [ ]:
len(final2)

In [ ]:
# final2[0]

In [ ]:
final2[200]['label']

 note to self, is it possible to then run lstm on only the body parts selected, like the legs and hips etc

#Combine the Two Versions

In [ ]:
coord2_df = pd.DataFrame(final2)

In [ ]:
coord2_df[frame_length_listdf] = pd.DataFrame(coord2_df.coords.tolist(), index= coord2_df.index)

In [ ]:
coord2_df = coord2_df.replace(['R','L'],['L', 'R'])

In [ ]:
main_df = pd.concat([coord_df,coord2_df])

In [ ]:
main_df = main_df.reset_index(drop=True)

In [ ]:
len(main_df)

In [ ]:
main_df.head()

In [ ]:
main_df['label'].value_counts()

# Normalizing the Pose Estimation Coords

In [ ]:
frame_len_list = ['F{}'.format(i) for i in range(1, frames_changable+1)]

In [ ]:
normalized_main_df = pd.DataFrame(columns = frame_len_list)

In [ ]:
normalized_main_df

In [ ]:
main_nest = []

In [ ]:
def normalize_coordinates(row_i, col_j):
    num_rows, num_cols = [567,960]
    x = col_j/(num_cols - 1.)
    y = row_i/(num_rows - 1.)
    return x, y

In [ ]:
# position shifting dataframe
for i in range(len(main_df)):
    ind_fr_list = []
    for y in range(3,frames_changable+3):
        test_skeli = main_df.iloc[[i],[y]].values
        test_skeli = np.array(test_skeli[0][0])
        test_skeli = [test_skeli[i:i+2] for i in range(0, len(test_skeli), 2)]
        test_skeli = np.array(test_skeli)
        y,x = test_skeli.T

        # 2 (Right?)
        RS = test_skeli[1]
        Rx, Ry = RS
        # 5 (Left?)
        LS = test_skeli[4]
        Lx, Ly = LS

        PSx = x - (Lx+Rx)/2
        PSy = y - (Ly+Ry)/2

        Sx = PSx/(960 - 1.)
        Sy = PSy/(567 - 1.)

        PS_coord = np.vstack((Sx,Sy)).T
        PS_coord = list(PS_coord)
        ind_fr_list.append(PS_coord)
   
    main_nest.append(ind_fr_list)
    normalized_main_df.loc[len(normalized_main_df)] = ind_fr_list

In [ ]:
main_nest_arr = np.array(main_nest)

In [ ]:
main_nest_arr = np.reshape(main_nest_arr, (791,20,34))

In [ ]:
main_nest_arr.shape

In [ ]:
labels = list(main_df['label'])

In [ ]:
dummy_labels = []

# append the activity encoder to dictionary and list but reverse left and right
for a in range(len(labels)):
   # ['center', 'left', 'right']
  if labels[a] == 'R': #0
    dummy_labels.append([0,0,1])
  elif labels[a] == 'C': #2
    dummy_labels.append([0,1,0])
  elif labels[a] == 'L': #1
    dummy_labels.append([1,0,0])


In [ ]:
unq = set(labels)

In [ ]:
unq

# LSTM preprocessing

In [ ]:
y = array(dummy_labels)

In [ ]:
X = main_nest_arr
x = main_nest_arr

#Cross Validation Split


In [ ]:
x.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
y_train[0]

In [ ]:
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]

In [ ]:
n_outputs

#

#Single Run LSTM

In [ ]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Flatten, ConvLSTM2D, BatchNormalization
from tensorflow.keras import Input
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.layers import RepeatVector

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import statistics
from tensorflow import keras


In [ ]:
model = Sequential()
model.add(LSTM(48, input_shape=(20,34)))
model.add(Dropout(.4))

model.add(Dense(3, activation='softmax'))

# adagrad = keras.optimizers.Adagrad(learning_rate=0.001)
# rms = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08,)
# sgd = keras.optimizers.SGD(learning_rate=0.001, clipnorm=1.)
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=30, validation_split=0.3) # 150 epochs

In [ ]:
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(x_test, y_test)

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax (y_pred, axis = 1)

# # 0 = L, 1 = C, 2 = R
y_test = np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred , normalize='pred')
print(result)

In [ ]:
print(classification_report(y_test, y_pred, digits=3))

In [ ]:
import random
randomlist = []
for i in range(0,159):
  n = random.randint(0,2)
  randomlist.append(n)
print(randomlist)

In [ ]:
print(classification_report(y_test, randomlist, digits=3))

# Keras Tuner 

In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import keras_tuner
from keras_tuner.engine.hyperparameters import HyperParameters
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import keras.backend as K
from keras.callbacks import EarlyStopping
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch


In [ ]:
def build_model1(hp):
    early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)
    model = Sequential()
    model.add(LSTM(hp.Int('input_unit',min_value=16,max_value=64,step=16),input_shape=(x_train.shape[1],x_train.shape[2])))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(y_train.shape[1], activation=hp.Choice('dense_activation',values=['relu', 'sigmoid', 'softmax'],default='softmax')))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    rms = keras.optimizers.RMSprop(learning_rate=hp_learning_rate, rho=0.9, epsilon=1e-08,)
    model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
    return model

In [ ]:
tuner1= RandomSearch(
        build_model1,
        objective='accuracy',
        max_trials=10,
        executions_per_trial=1
        )

In [ ]:
tuner1.search(
        x=x_train,
        y=y_train,
        epochs=100,
        validation_split=0.25)

In [ ]:
best_model = tuner1.get_best_models(num_models=1)[0]

In [ ]:
best_model.summary()

In [ ]:
best_hps = tuner1.get_best_hyperparameters(5)

In [ ]:
model = build_model(best_hps[0])

In [ ]:
x_all = np.concatenate((x_train[:158], x_train[159:]))
y_all = np.concatenate((y_train[:158], y_train[159:]))
model.fit(x=x_all, y=y_all, epochs=100)

In [ ]:
tuner1.results_summary()

In [ ]:
y_pred = best_model.predict(x_test)

In [ ]:
y_pred.shape

#Multiple Runs LSTM (50 runs)

In [ ]:
from sklearn.metrics import accuracy_score
import statistics
from tensorflow import keras
import numpy as np
import seaborn as sns


scores = list()
for r in range(50):
  x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
  model = Sequential()
  model.add(LSTM(48,  input_shape=(20,34)))
  model.add(Dropout(.4))
  model.add(Dense(3, activation='softmax'))

  # rms = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08,)
  # sgd = keras.optimizers.SGD(learning_rate=0.001, clipnorm=1.)
  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  model.fit(x_train, y_train, epochs=30, validation_split=0.25)

  y_pred = model.predict(x_test)
  y_pred = np.argmax(y_pred, axis=1)
  y_test = np.argmax(y_test, axis=1)
  score = accuracy_score(y_test, y_pred)
  scores.append(score)
print(statistics.mean(scores))
print(scores)

In [ ]:
# scores

In [ ]:
sns.set_style('whitegrid')
sns.kdeplot(np.array(scores), bw=0.5)

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax (y_pred, axis = 1)

# 0 = L, 1 = C, 2 = R
y_test = np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred , normalize='pred')
print(result)

In [ ]:
print(result)

In [ ]:
print(classification_report(y_test, y_pred, digits=3))

# Multiple Random Runs (50 runs)

In [ ]:
scores = list()
for r in range(50):
  x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
  randomlist = []
  for i in range(0,159):
    n = random.randint(0,2)
    randomlist.append(n)
  
  y_test = np.argmax(y_test, axis=1)
  score = accuracy_score(y_test, randomlist)
  scores.append(score)
print(statistics.mean(scores))
print(scores)

In [ ]:
scores

In [ ]:
result = confusion_matrix(y_test, randomlist , normalize='pred')
print(result)

In [ ]:
sns.set_style('whitegrid')
sns.kdeplot(np.array(scores), bw=0.5)